In [ ]:
!pip install kagglehub dask xgboost==1.7.6  scikit-learn==1.3.2 distributed dask-xgboost dask-ml[complete]  "bokeh>=3.1.0" pyngrok dask[distributed]

In [ ]:
from dask.distributed import Client
import time
import pandas as pd
import glob
import kagglehub
from pyngrok import ngrok
import pandas as pd
import dask.dataframe as dd
from dask.dataframe import map_partitions
import dask.array as da
from sklearn.model_selection import train_test_split
import time
from xgboost.dask import DaskXGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [ ]:
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")
print("Path to dataset files:", path)

100%|██████████| 1.78G/1.78G [00:12<00:00, 154MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2


In [ ]:
#!ngrok authtoken xxxxxxxxxxx
client = Client(n_workers=8, threads_per_worker=2)
print(client)

#public_url = ngrok.connect(8787)
#print(f"Dask Dashboard is available at: {public_url}")

start_time = time.time()

local_path = "/root/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2/*.csv"
#csv_files = glob.glob(local_path)

df = dd.read_csv(local_path, assume_missing=True)
#df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
df = df.drop(columns=["RateCodeID"], errors="ignore")
df = df.repartition(npartitions=12)

print("Columns:", df.columns)
print(df.head())

In [ ]:
meta = {
    "VendorID": "Int64",
    "tpep_pickup_datetime": "datetime64[ns]",
    "tpep_dropoff_datetime": "datetime64[ns]",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "pickup_longitude": "float64",
    "pickup_latitude": "float64",
    "store_and_fwd_flag": "object",
    "dropoff_longitude": "float64",
    "dropoff_latitude": "float64",
    "payment_type": "Int64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "pickup_hour": "Int64",
    "day_of_week": "Int64",
    "is_weekend": "Int64",
    "trip_duration": "float64",
}

In [ ]:
def process_data(df: pd.DataFrame):
    # 1 Filter invalid trip_distance:
    df = df[(df["trip_distance"] != 0) & (df["trip_distance"].notna())]
    df = df[(df["tpep_pickup_datetime"] != 0) & (df["tpep_pickup_datetime"].notna())]
    df = df[(df["tpep_dropoff_datetime"] != 0) & (df["tpep_dropoff_datetime"].notna())]

    # 2 Outlier Handling:
    trip_distance_quantiles = df["trip_distance"].quantile([0.01, 0.99])
    fare_amount_quantiles = df["fare_amount"].quantile([0.01, 0.99])
    df = df[
        (df["trip_distance"] >= trip_distance_quantiles[0.01])
        & (df["trip_distance"] <= trip_distance_quantiles[0.99])
        & (df["fare_amount"] >= fare_amount_quantiles[0.01])
        & (df["fare_amount"] <= fare_amount_quantiles[0.99])
    ]

    # 3 Datetime Features:
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], errors='coerce')
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors='coerce')

    df[['pickup_hour','day_of_week']] = pd.DataFrame(df['tpep_pickup_datetime'].apply(lambda x: [x.hour, x.dayofweek]).tolist(), index= df.index)
    df.loc[:, "is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    # 4 Trip Duration:
    df.loc[:, "trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60.0

    # 5 Filter for Valid Fare Amounts:
    df = df[df["fare_amount"] > 0]

    df = df.dropna(subset=["pickup_hour", "is_weekend", "trip_duration", "fare_amount", "trip_distance"])


    return df

In [ ]:
df = df.map_partitions(process_data, meta=meta)
df.compute()

INFO:distributed.core:Event loop was unresponsive in Nanny for 6.07s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 6.06s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 6.06s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 6.06s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 6.06s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeout

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_hour,day_of_week,is_weekend,trip_duration
0,2.0,2015-01-15 19:05:39,2015-01-15 19:23:42,1.0,1.59,-73.993896,40.750111,N,-73.974785,40.750618,...,1.0,0.5,3.25,0.0,0.3,17.05,19,3,0,18.050000
1,1.0,2015-01-10 20:33:38,2015-01-10 20:53:28,1.0,3.30,-74.001648,40.724243,N,-73.994415,40.759109,...,0.5,0.5,2.00,0.0,0.3,17.80,20,5,1,19.833333
2,1.0,2015-01-10 20:33:38,2015-01-10 20:43:41,1.0,1.80,-73.963341,40.802788,N,-73.951820,40.824413,...,0.5,0.5,0.00,0.0,0.3,10.80,20,5,1,10.050000
3,1.0,2015-01-10 20:33:39,2015-01-10 20:35:31,1.0,0.50,-74.009087,40.713818,N,-74.004326,40.719986,...,0.5,0.5,0.00,0.0,0.3,4.80,20,5,1,1.866667
4,1.0,2015-01-10 20:33:39,2015-01-10 20:52:58,1.0,3.00,-73.971176,40.762428,N,-74.004181,40.742653,...,0.5,0.5,0.00,0.0,0.3,16.30,20,5,1,19.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420265,1.0,2016-03-29 21:28:47,2016-03-29 21:36:18,1.0,1.50,-73.937790,40.748898,Y,-73.920471,40.739483,...,0.5,0.5,0.00,0.0,0.3,8.80,21,1,0,7.516667
420268,1.0,2016-03-31 13:41:59,2016-03-31 13:59:01,1.0,1.50,-73.986847,40.761150,Y,-73.967972,40.765274,...,0.0,0.5,0.00,0.0,0.3,12.30,13,3,0,17.033333
420271,1.0,2016-03-20 08:59:21,2016-04-18 10:58:05,1.0,12.10,-73.789154,40.646736,N,-73.875351,40.757648,...,0.0,0.5,5.00,0.0,0.3,40.30,8,6,1,41878.733333
420273,1.0,2016-03-26 03:02:32,2016-06-14 18:47:55,1.0,0.40,-73.977356,40.774471,N,-73.982536,40.772408,...,1.0,0.5,1.70,0.0,0.3,7.50,3,5,1,116145.383333


In [ ]:
X = df[["pickup_hour", "passenger_count", "is_weekend", "trip_duration", "trip_distance"]]
y = df["fare_amount"]

In [ ]:
len(df)

45001644

In [ ]:
X.head(5)

,pickup_hour,passenger_count,is_weekend,trip_duration,trip_distance
0,19,1.0,0,18.050000,1.59
1,20,1.0,1,19.833333,3.30
2,20,1.0,1,10.050000,1.80
3,20,1.0,1,1.866667,0.50
4,20,1.0,1,19.316667,3.00


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X.compute(), y.compute(), test_size=0.2, random_state=42
)
#X_train_dask = da.from_array(X_train.values, chunks=(100000, X_train.shape[1]))
#y_train_dask = da.from_array(y_train.values, chunks=(100000,))

In [ ]:
X_train_dask = dd.from_pandas(X_train, npartitions=12)
X_test_dask = dd.from_pandas(X_test, npartitions=12)
y_train_dask = dd.from_pandas(y_train, npartitions=12)
y_test_dask = dd.from_pandas(y_test, npartitions=12)

In [ ]:
model = DaskXGBRegressor(
    objective='reg:squarederror',
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    eval_metric=["rmse", "mae"],
    random_state=42,
    n_estimators=50
)

In [ ]:
model.fit(X_train_dask, y_train_dask)

In [ ]:
y_pred_dask = model.predict(X_test_dask)

# Calculate MAE and RMSE
y_pred = y_pred_dask.compute()
y_test = y_test_dask.compute()
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")